

## Kubernetes Lab



In [ ]:
. ../NB_bash_functions.rc

In [ ]:
cd /home/mjb/MINIKUBE
#ls -altr

In [ ]:
. minikube.rc

In [ ]:
which docker; docker version

In [ ]:
which minikube; minikube version

In [ ]:
which kubectl; kubectl version

In [ ]:
minikube status

In [ ]:
minikube delete

In [ ]:
#. minikube.rc

# 0. Kubernetes Cluster Creation

Using the minikube tool we can create a single-node Kubernetes cluster.

The cluster runs within a VirtualBox VM running the boot2docker.iso image.

To use this cluster we need just 3 executables:
- minikube itself
- the docker client
- the kubectl client

In [ ]:
which minikube; which docker; which kubectl
ls -altrh $(which minikube)
#echo $PATH

In [ ]:
docker version; minikube version

minikube should not be already running:

In [ ]:
minikube status

In [ ]:
time minikube start

kubectl version

Once the cluster is started we can obtain the environment variables needed to allow our docker client to communicate with the cluster.

In [ ]:
minikube docker-env

In [ ]:
eval $(minikube docker-env); docker version

Running ```docker ps``` we can see the containers used to implement the cluster.

It may take a minute or so for all 4 containers to be running:

In [ ]:
docker ps 

We can also connect to the node (VM) on which the cluster is running by using the command

```minikube ssh```

In [ ]:
minikube ssh "uptime; echo; docker version; echo; hostname"

Let's first cleanup any Pods, Services, Deployments whch may be running

(there should be none if we just started the cluster).

In [ ]:
#bash -x ./cleanup.sh

In [ ]:
kubectl get nodes

In [ ]:
kubectl get pods

In [ ]:
kubectl get service

In [ ]:
kubectl get deployments

In [ ]:
kubectl get namespaces

# Demo start

# 1. Creating a Cluster with minikube

The minikube executable is a tool created by the kubernetes project for performing demos/tutorials of a single-node kubernetes cluster.

# kubectl

kubectl is the kubernetes tool used for managing a cluster from the command-line.
It is based on the kubernetes API.

kubectl commands are of the form:

    kubectl <verb> <noun>
    
e.g.

    kubectl get nodes

to see what nodes exist in the cluster.

In [ ]:
kubectl get nodes

Note that abbreviations exist, e.g. no for nodes

## 1. kubectl commands

We can get a list of available kubectl commands (verbs) just by typing kubectl

In [ ]:
kubectl

Similarly we can get a list of subcommands (nouns) to which they can be applied.

```kubectl get```

will show us what items we can 'get':

In [ ]:
kubectl get

NB_continue

In [ ]:
kubectl describe

NB_continue

In [ ]:
kubectl get nodes

In [ ]:
kubectl describe nodes

In [ ]:
kubectl get pods

In [ ]:
kubectl describe pods

We can get detailed help on the "get" command:

In [ ]:
which kubectl; which minikube

In [ ]:
kubectl version

In [ ]:
kubectl cluster-info

## Opening the kubernetes dashboard

From the command-line type the command:

```minikube dashboard```

In [ ]:
# minikube dashboard

In [ ]:
NB_kube_loop_until_dashboard_url_available

# 2. Creating a Pod/Service/Deployment


### Kubernetes Deployments
Once you have a running Kubernetes cluster, you can deploy your containerized applications on top of it. To do so, you create a Kubernetes Deployment. The Deployment is responsible for creating and updating instances of your application. Once you've created a Deployment, the Kubernetes master schedules the application instances that the Deployment creates onto individual Nodes in the cluster.

Once the application instances are created, a Kubernetes Deployment Controller continuously monitors those instances. The Deployment controller replaces an instance if the Node hosting it goes down or it is deleted. This provides a self-healing mechanism to address machine failure or maintenance.

In a pre-orchestration world, installation scripts would often be used to start applications, but they did not allow recovery from machine failure. By both creating your application instances and keeping them running across Nodes, Kubernetes Deployments provide a fundamentally different approach to application management.


You can create and manage a Deployment by using the Kubernetes command line interface, Kubectl. Kubectl uses the Kubernetes API to interact with the cluster. In this module, you'll learn the most common Kubectl commands needed to create Deployments that run your applications on a Kubernetes cluster.

When you create a Deployment, you'll need to specify the container image for your application and the number of replicas that you want to run. You can change that information later by updating your Deployment; Modules 5 and 6 of the bootcamp discuss how you can scale and update your Deployments.

Applications need to be packaged into one of the supported container formats in order to be deployed on Kubernetes

For our first Deployment, we'll use a Node.js application packaged in a Docker container. The source code and the Dockerfile are available in the GitHub repository for the Kubernetes Bootcamp.

Now that you know what Deployments are, let's go to the online tutorial and deploy our first app!



Inspired by https://media-glass.es/launching-a-local-kubernetes-lab-using-minikube-39560f792889#.3h0n2dh3f

## 2.1 Deploying an app


In [ ]:
kubectl run my-nginx --image=nginx --replicas=2 --port=80

It may take some time for the pods to start as the node downloads the nginx image for the first time

In [ ]:
date; echo
kubectl get pods | NB_highlight STATUS

NB_kube_loop_while_pods_creating
echo; date

After a few minutes the pods are "READY":

In [ ]:
kubectl get pods | NB_highlight STATUS

In [ ]:
kubectl describe deploy my-nginx | NB_highlight available

In [ ]:
NB_kube_loop_until_available deploy my-nginx

## 2.2 Accessing the app via kube-proxy

By default deployed applications are visible only inside the Kubernetes cluster. Exposing our application externally will be covered in Module 4. To view the application output without exposing it externally, we’ll create a route between our terminal and the Kubernetes cluster using a proxy:

By default deployed applications are visible only inside the Kubernetes cluster. Exposing our application externally will be covered in Module 4. To view the application output without exposing it externally, we’ll create a route between our terminal and the Kubernetes cluster using a proxy:

**In another terminal window launch**:
```
    kubectl proxy```
    
We now have a connection between our host (the online terminal) and the Kubernetes cluster. The started proxy enables direct access to the API. The app runs inside a Pod (we'll cover the Pod concept in next module). Get the name of the Pod and store it in the POD_NAME environment variable:


In [ ]:
# In another window run
#    kubectl proxy

export POD_NAMES=$(kubectl get pods -o go-template --template '{{range .items}}{{.metadata.name}}{{"\n"}}{{end}}')
echo Names of the Pods: $POD_NAMES

In [ ]:
for POD_NAME in $POD_NAMES; do
    echo; echo "-- POD $POD_NAME: ------"
    echo "curl http://localhost:8001/api/v1/proxy/namespaces/default/pods/$POD_NAME/"
    
    curl http://localhost:8001/api/v1/proxy/namespaces/default/pods/$POD_NAME/
    #break
done

## 2.3 Service creation from kubectl


Exposing our deployment with type NodePort to make it available as a service on each cluster node

In [ ]:
kubectl expose deployment my-nginx --type=NodePort

In [ ]:
kubectl get pods

In [ ]:
kubectl get deploy

In [ ]:
kubectl get all #--all-namespaces

In [ ]:
kubectl describe deploy

Now let's obtain the url of our 'my-nginx' service

In [ ]:
minikube service my-nginx --url

In [ ]:
kubectl describe deploy | NB_highlight available

In [ ]:
minikube service my-nginx --url

#open $(minikube service my-nginx --url)

We can now access this url from our host, using curl or a web-browser:

In [ ]:
curl $(minikube service my-nginx --url)

# 2.4 Rolling upgrade of a service

In this example we will create a new service and show how we can easily perform a rolling upgrade to a new version of the service.

We will also see how we can roll back to the previous service version.


In [ ]:
kubectl get --namespace default all

In [ ]:
#kubectl  delete deploy k8s-demo
#kubectl delete service k8s-demo-service

In [ ]:
kubectl run k8s-demo --labels="owner=mike" --port 8080 --replicas=3 --image mjbright/k8s-demo:1

In [ ]:
kubectl get --namespace default all

In [ ]:
kubectl describe deploy k8s-demo

In [ ]:
kubectl describe replicaset k8s-demo

In [ ]:
EXPOSE_TYPE=LoadBalancer
#EXPOSE_TYPE=NodePort
EPORT=8080

kubectl expose deploy k8s-demo --type=$EXPOSE_TYPE --name=k8s-demo-service --port $EPORT

In [ ]:
kubectl describe nodes | grep Addresses:

kubectl describe nodes | awk '/Addresses:/ { FS=","; $0=$2; print $1; }'

NODEIP=$(kubectl describe nodes | awk '/Addresses:/ { FS=","; $0=$2; print $1; }')

In [ ]:
kubectl get            service

In [ ]:
kubectl get service | awk '/k8s-demo-service/ { FS=":"; $0=$4; FS="/"; $0=$2; print $1; }'

PORT=$(kubectl get service | awk '/k8s-demo-service/ { FS=":"; $0=$4; FS="/"; $0=$2; print $1; }')

SERVICE_URL="http://${NODEIP}:${PORT}"

In [ ]:
wget -O - http://${NODEIP}:${PORT}

In [ ]:
echo $SERVICE_URL

In [ ]:
show_access_ports() {
    echo "------------------------------------------------"                                                              
    echo "Access to pods directly using:"                                                                                
    kubectl describe pods | awk '/IP:/ { print "minikube ssh curl "$2":8080"; }'                                                      
                                                                                                                         
    echo                                                                                                                 
    echo "Access service by running command:"                                                                            
    echo "    curl $SERVICE_URL"                                                                                         
}

show_access_ports

In [ ]:
kubectl describe pods

In [ ]:
~/MINIKUBE/bin_0.22.3/minikube ssh curl 172.17.0.6:8080

In [ ]:
~/MINIKUBE/bin_0.22.3/minikube ssh curl 172.17.0.7:8080

### Now let us perform a rolling upgrade of our service

We will now upgrade to version 2 of our application.

Note that it is the 'deploy' object which corresponds to the version of an application

In [ ]:
kubectl set image deployment k8s-demo k8s-demo=mjbright/k8s-demo:2 --record

Now let us observe the rollout as it happens:

In [ ]:
kubectl rollout status deploy k8s-demo

In [ ]:
kubectl get --namespace default all

In [ ]:
show_access_ports

In [ ]:
minikube ssh curl 172.17.0.6:8080

In [ ]:
minikube ssh curl 172.17.0.9:8080

In [ ]:
kubectl get  --selector "owner=mike" service

# 3. Viewing Pods

In [ ]:
kubectl get pods

In [ ]:
kubectl describe pods

In [ ]:
kubectl get pods -L NAME

In [ ]:
kubectl get pods -o go-template --template '{{range .items}}{{.metadata.name}}{{"\n"}}{{end}}'

In [ ]:
POD_NAME=$(kubectl get pods -o go-template --template '{{range .items}}{{.metadata.name}}{{"\n"}}{{end}}' | tail -1)

In [ ]:
kubectl logs $POD_NAME

In [ ]:
kubectl exec $POD_NAME env

In [ ]:
kubectl exec $POD_NAME hostname

In [ ]:
kubectl exec $POD_NAME ls /sbin

We can see the ip address of a pod (same ip address for all containers in that pod):

In [ ]:
kubectl exec $POD_NAME -- hostname -i

In [ ]:
kubectl get pod $POD_NAME

In [ ]:
kubectl get pod $POD_NAME -o yaml

# 4. SKIP - Inspecting our service

A Service provides load balancing of traffic across the contained set of Pods. This is useful when a service is created to group all Pods from a specific Deployment (our application will make use of this in the next module, when we'll have multiple instances running).

Services are also responsible for service-discovery within the cluster (covered in Accessing the Service). This will for example allow a frontend service (like a web server) to receive traffic from a backend service (like a database) without worrying about Pods.

Services match a set of Pods using Label Selectors, a grouping primitive that allows logical operation on Labels.

Labels are key/value pairs that are attached to objects, such as Pods and you can think of them as hashtags from social media. They are used to organize related objects in a way meaningful to the users like:

Production environment (production, test, dev)
Application version (beta, v1.3)
Type of service/server (frontend, backend, database)
Labels are key/value pairs that are attached to objects


We have a Service called kubernetes that is created by default when minikube starts the cluster. To create a new service and expose it to external traffic we’ll use the expose command with NodePort as parameter (minikube does not support the LoadBalancer option yet)

In [ ]:
kubectl get svc

In [ ]:
kubectl get services/my-nginx

We can ssh into our minikube node to find it's eth1 ip address.
This is the address we will use to access our service (the address used for $DOCKER_HOST):

In [ ]:
minikube ssh ip a show dev eth1

In [ ]:
echo $DOCKER_HOST

In [ ]:
HOST_IP=$(echo $DOCKER_HOST | sed -e 's/.*:\/\///' -e 's/:.*//')
echo HOST_IP=$HOST_IP

We can then use the describe service command to see which port (NodePort) is to be used:

In [ ]:
kubectl describe svc my-nginx

So now we know on which port of our node we can access our service:

Let's access the port automatically

In [ ]:
export NODE_PORT=$(kubectl get services/my-nginx -o go-template='{{(index .spec.ports 0).nodePort}}')
echo NODE_PORT=$NODE_PORT

#export NODE_IP=$(kubectl get services/my-nginx -o go-template='{{(index .spec.clusterIP)}}')
#echo NODE_IP=$NODE_IP

In [ ]:
echo curl http://${HOST_IP}:${NODE_PORT}
curl http://${HOST_IP}:${NODE_PORT}

In [ ]:
kubectl describe deploy

# 5. Accessing objects using label selectors

We saw above that all my-nginx pods have the label run=my-nginx.

We can use this to select only those pods:

In [ ]:
kubectl get pods -l run=my-nginx

#### Setting labels

We can also set labels on any object.

Let's set a label '*app=v1*' on just one of our nginx pods:


In [ ]:
kubectl label pod $POD_NAME app=v1

Now running describe pod, we see both pods but only one of them has the '*app=v1*' label:

In [ ]:
kubectl describe pod

Now we can select just pods with '*app=v1*':

In [ ]:
kubectl get pod -l app=v1

### We will now delete the my-nginx service, we will re-create in the next chapter

In [ ]:
kubectl delete service -l run=my-nginx

In [ ]:
kubectl get service

# 4. Scaling an app

In [ ]:
kubectl scale deployments/my-nginx --replicas=4

In [ ]:
kubectl get deploy

In [ ]:
kubectl get deploy

In [ ]:
kubectl get pods -o wide

In [ ]:
kubectl describe deploy/my-nginx

In [ ]:
kubectl describe services/my-nginx;

NB_continue

In [ ]:
kubectl expose deployment my-nginx --type=NodePort

In [ ]:
kubectl describe services/my-nginx

In [ ]:
export NODE_PORT=$(kubectl get services/my-nginx -o go-template='{{(index .spec.ports 0).nodePort}}')
echo NODE_PORT=$NODE_PORT

HOST_IP=$(echo $DOCKER_HOST | sed -e 's/.*:\/\///' -e 's/:.*//')

In [ ]:
curl $HOST_IP:$NODE_PORT

In [ ]:
kubectl scale deployments/my-nginx --replicas=2

In [ ]:
kubectl get pods -o wide

In [ ]:
sleep 10; kubectl get pods -o wide

In [ ]:
kubectl get deploy

# 5. Performing a Rolling Update

Updating an application
Users expect applications to be available all the time and developers are expected to deploy new versions of them several times a day. In Kubernetes this is done with rolling updates. Rolling updates allow Deployments' update to take place with zero downtime by incrementally updating Pods instances with new ones. The new Pods will be scheduled on Nodes with available resources.

In the previous module we scaled our application to run multiple instances. This is a requirement for performing updates without affecting application availability. By default, the maximum number of Pods that can be unavailable during the update and the maximum number of new Pods that can be created, is one. Both options can be configured to either numbers or percentages (of Pods). In Kubernetes, updates are versioned and any Deployment update can be reverted to previous (stable) version.

Similar to application Scaling, If a Deployment is exposed publicly, the Service will load-balance the traffic only to available Pods during the update. An available Pod is an instance that is available to the users of the application.

Rolling updates allow the following actions:

- Promote an application from one environment to another (via container image updates)
- Rollback to previous versions
- Continuous Integration and Continuous Delivery of applications with zero downtime


In [ ]:
kubectl run kubernetes-demo --image=docker.io/mjbright/k8s-demo:1 --port=8080

echo;
kubectl get deployments

Wait until the pod is deployed - may need to download the *kubernetes-demo* image:

In [ ]:
kubectl get pods

In [ ]:
NB_kube_loop_while_pods_creating

In [ ]:
kubectl get pods

Now we can expose our deployment as a service

In [ ]:
kubectl expose deployment kubernetes-demo --type=NodePort --port=8080

In [ ]:
kubectl describe services/kubernetes-demo

Let's obtain the NODE_PORT so we can access the new '*docker-demo*' service:

In [ ]:
kubectl get services/kubernetes-demo -o go-template='{{(index .spec.ports 0).nodePort}}'

#NB_pause

In [ ]:
NB_kube_loop_until_available deploy kubernetes-demo

In [ ]:
NB_kube_loop_until_available services kubernetes-demo

In [ ]:
export NODE_PORT=$(kubectl get services/kubernetes-demo -o go-template='{{(index .spec.ports 0).nodePort}}')
echo NODE_PORT=$NODE_PORT

HOST_IP=$(echo $DOCKER_HOST | sed -e 's/.*:\/\///' -e 's/:.*//')
echo HOST_IP=$HOST_IP

In [ ]:
echo curl http://${HOST_IP}:${NODE_PORT}
curl $HOST_IP:$NODE_PORT


### Perform the rolling update from v1 to v2

In [ ]:
kubectl set image deployments/kubernetes-demo kubernetes-demo=mjbright/k8s-demo:2

In [ ]:
kubectl rollout status deployments/kubernetes-demo

Using '*kubectl describe*' we can see that initially when we created the deployment (1st event below) that version of '*kubernetes-demo*' was scaled to 1.

Then when we performed the rollout we see the new version of '*kubernetes-demo*' was scaled to 1 and once this was achieved, the old version was scaled back to 0, disabling the old version.

In [ ]:
kubectl describe deployments/kubernetes-demo

In [ ]:
kubectl rollout status deployments/kubernetes-demo

In [ ]:
kubectl describe deployments/kubernetes-demo

... now accessing the service we can see that the version (v=2) has changed

In [ ]:
NB_kube_loop_until_available deploy kubernetes-demo

In [ ]:

NB_curl_until_OK http://$HOST_IP:$NODE_PORT

In [ ]:
NB_time_taken